In [ ]:
# Make sure that you have set up API KEYS in Secrets on the left menu icon.
from google.colab import userdata
openai_api_key = userdata.get('OPENAI_API_KEY')
tavily_api_key = userdata.get('TAVILY_API_KEY')

In [ ]:
%pip install -q langchain openai langchain-openai faiss-gpu langchain-community

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import create_openai_functions_agent, AgentExecutor
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import HumanMessage, AIMessage
from langchain.tools.retriever import create_retriever_tool

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores.faiss import FAISS

Create Retriever

In [ ]:
loader = WebBaseLoader("https://python.langchain.com/docs/expression_language/")
docs = loader.load()

splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=20
)
splitDocs = splitter.split_documents(docs)

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = openai_api_key
embedding = OpenAIEmbeddings( )
vectorStore = FAISS.from_documents(docs, embedding=embedding)
retriever = vectorStore.as_retriever(search_kwargs={"k": 3})

In [ ]:
model = ChatOpenAI(
    model='gpt-3.5-turbo-1106',
    temperature=0.7,
    openai_api_key=openai_api_key
)

In [ ]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a friendly assistant called Max."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

In [ ]:
import os
os.environ["TAVILY_API_KEY"] = tavily_api_key
search = TavilySearchResults()
retriever_tools = create_retriever_tool(
    retriever,
    "lcel_search",
    "Use this tool when searching for information about Langchain Expression Language (LCEL)."
)
tools = [search, retriever_tools]

In [ ]:
agent = create_openai_functions_agent(
    llm=model,
    prompt=prompt,
    tools=tools
)

In [ ]:
agentExecutor = AgentExecutor(
    agent=agent,
    tools=tools
)

In [ ]:
def process_chat(agentExecutor, user_input, chat_history):
    response = agentExecutor.invoke({
        "input": user_input,
        "chat_history": chat_history
    })
    return response["output"]

In [ ]:
def add_user_message(chat_history, content):
    chat_history.append({"role": "user", "content": content})

def add_assistant_message(chat_history, content):
    chat_history.append({"role": "assistant", "content": content})

import textwrap

def print_assistant_message(message):
    wrapped_message = textwrap.fill(message, width=80)  # Adjust the width as needed
    print("Assistant:", wrapped_message)

def main():
    chat_history = []

    while True:
        user_input = input("You: ")
        if user_input.lower() == "quit":
            break

        add_user_message(chat_history, user_input)
        output = process_chat(agentExecutor, user_input, chat_history)
        add_assistant_message(chat_history, output)
        print_assistant_message(output)

if __name__ == "__main__":
    main()